In [1]:
#This is an implementation using Keras. It is maybe a very simple and readable implementation.  
#To read more look at these References:

#https://robintibor.github.io/braindecode/

#https://www.tensorflow.org/alpha/tutorials/keras/basic_classification/

#Feel free to contact me if needed:
#https://github.com/MohammadJavadD
#https://www.linkedin.com/in/mjdarvishi/

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

1.13.1


In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3935876404748300791
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 9121682555
locality {
  bus_id: 1
  links {
  }
}
incarnation: 17647232605841408151
physical_device_desc: "device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5"
]


In [4]:
!pip install braindecode

In [5]:
import logging
import importlib
importlib.reload(logging) # see https://stackoverflow.com/a/21475297/1469195
log = logging.getLogger()
log.setLevel('INFO')
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)

In [6]:
import mne
import numpy as np
from mne.io import concatenate_raws
from braindecode.datautil.signal_target import SignalAndTarget

# First 55 subjects as train and validation
physionet_paths = [ mne.datasets.eegbci.load_data(sub_id,[4,8,12,]) for sub_id in range(1,56)]
physionet_paths = np.concatenate(physionet_paths)
parts = [mne.io.read_raw_edf(path, preload=True,stim_channel='auto')
         for path in physionet_paths]

raw = concatenate_raws(parts)

picks = mne.pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False,
                   exclude='bads')

#events = mne.find_events(raw, shortest_event=0, stim_channel='STI 014')
# Find the events in this dataset
events, _ = mne.events_from_annotations(raw)

# Read epochs (train will be done only between 1 and 2s)
# Testing will be done with a running classifier
epoched = mne.Epochs(raw, events, dict(hands=2, feet=3), tmin=1, tmax=4.1, proj=False, picks=picks,
                baseline=None, preload=True)

# 51-55 as validation subjects
'''
physionet_paths_valid = [mne.datasets.eegbci.load_data(sub_id,[4,8,12,]) for sub_id in range(51,56)]
physionet_paths_valid = np.concatenate(physionet_paths_valid)
parts_valid = [mne.io.read_raw_edf(path, preload=True,stim_channel='auto')
         for path in physionet_paths_valid]
raw_valid = concatenate_raws(parts_valid)

picks_valid = mne.pick_types(raw_valid.info, meg=False, eeg=True, stim=False, eog=False,
                   exclude='bads')

#events_valid = mne.find_events(raw_valid, shortest_event=0, stim_channel='STI 014')
# Find the events in this dataset
events_valid, _ = mne.events_from_annotations(raw_valid)

# Read epochs (train will be done only between 1 and 2s)
# Testing will be done with a running classifier
epoched_valid = mne.Epochs(raw_valid, events_valid, dict(hands=2, feet=3), tmin=1, tmax=4.1, proj=False, picks=picks_valid,
                baseline=None, preload=True)

'''
# 55-65 as test subjects
physionet_paths_test = [mne.datasets.eegbci.load_data(sub_id,[4,8,12,]) for sub_id in range(56,65)]
physionet_paths_test = np.concatenate(physionet_paths_test)
parts_test = [mne.io.read_raw_edf(path, preload=True,stim_channel='auto')
         for path in physionet_paths_test]
raw_test = concatenate_raws(parts_test)

picks_test = mne.pick_types(raw_test.info, meg=False, eeg=True, stim=False, eog=False,
                   exclude='bads')

#events_test = mne.find_events(raw_valid, shortest_event=0, stim_channel='STI 014')
# Find the events in this dataset
events_test, _ = mne.events_from_annotations(raw_test)

# Read epochs (train will be done only between 1 and 2s)
# Testing will be done with a running classifier
epoched_test = mne.Epochs(raw_test, events_test, dict(hands=2, feet=3), tmin=1, tmax=4.1, proj=False, picks=picks_test,
                baseline=None, preload=True)

############################
train_X = (epoched.get_data() * 1e6).astype(np.float32)
train_y = (epoched.events[:,2] - 2).astype(np.int64) #2,3 -> 0,1
#valid_X = (epoched_valid.get_data() * 1e6).astype(np.float32)
#valid_y = (epoched_valid.events[:,2] - 2).astype(np.int64) #2,3 -> 0,1
test_X = (epoched_test.get_data() * 1e6).astype(np.float32)
test_y = (epoched_test.events[:,2] - 2).astype(np.int64) #2,3 -> 0,1

print(train_X.shape)
print(train_y.shape)

Extracting EDF parameters from C:\Users\MJD\mne_data\MNE-eegbci-data\physiobank\database\eegmmidb\S001\S001R04.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from C:\Users\MJD\mne_data\MNE-eegbci-data\physiobank\database\eegmmidb\S001\S001R08.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from C:\Users\MJD\mne_data\MNE-eegbci-data\physiobank\database\eegmmidb\S001\S001R12.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from C:\Users\MJD\mne_data\MNE-eegbci-data\physiobank\database\eegmmidb\S002\S002R04.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      

Extracting EDF parameters from C:\Users\MJD\mne_data\MNE-eegbci-data\physiobank\database\eegmmidb\S011\S011R12.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Extracting EDF parameters from C:\Users\MJD\mne_data\MNE-eegbci-data\physiobank\database\eegmmidb\S012\S012R04.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Extracting EDF parameters from C:\Users\MJD\mne_data\MNE-eegbci-data\physiobank\database\eegmmidb\S012\S012R08.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Extracting EDF parameters from C:\Users\MJD\mne_data\MNE-eegbci-data\physiobank\database\eegmmidb\S012\S012R12.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      

Extracting EDF parameters from C:\Users\MJD\mne_data\MNE-eegbci-data\physiobank\database\eegmmidb\S022\S022R08.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from C:\Users\MJD\mne_data\MNE-eegbci-data\physiobank\database\eegmmidb\S022\S022R12.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from C:\Users\MJD\mne_data\MNE-eegbci-data\physiobank\database\eegmmidb\S023\S023R04.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Extracting EDF parameters from C:\Users\MJD\mne_data\MNE-eegbci-data\physiobank\database\eegmmidb\S023\S023R08.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      

Extracting EDF parameters from C:\Users\MJD\mne_data\MNE-eegbci-data\physiobank\database\eegmmidb\S033\S033R04.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Extracting EDF parameters from C:\Users\MJD\mne_data\MNE-eegbci-data\physiobank\database\eegmmidb\S033\S033R08.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Extracting EDF parameters from C:\Users\MJD\mne_data\MNE-eegbci-data\physiobank\database\eegmmidb\S033\S033R12.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Extracting EDF parameters from C:\Users\MJD\mne_data\MNE-eegbci-data\physiobank\database\eegmmidb\S034\S034R04.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19839  =      

Extracting EDF parameters from C:\Users\MJD\mne_data\MNE-eegbci-data\physiobank\database\eegmmidb\S043\S043R12.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Extracting EDF parameters from C:\Users\MJD\mne_data\MNE-eegbci-data\physiobank\database\eegmmidb\S044\S044R04.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Extracting EDF parameters from C:\Users\MJD\mne_data\MNE-eegbci-data\physiobank\database\eegmmidb\S044\S044R08.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Extracting EDF parameters from C:\Users\MJD\mne_data\MNE-eegbci-data\physiobank\database\eegmmidb\S044\S044R12.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      

Extracting EDF parameters from C:\Users\MJD\mne_data\MNE-eegbci-data\physiobank\database\eegmmidb\S054\S054R08.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Extracting EDF parameters from C:\Users\MJD\mne_data\MNE-eegbci-data\physiobank\database\eegmmidb\S054\S054R12.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Extracting EDF parameters from C:\Users\MJD\mne_data\MNE-eegbci-data\physiobank\database\eegmmidb\S055\S055R04.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Extracting EDF parameters from C:\Users\MJD\mne_data\MNE-eegbci-data\physiobank\database\eegmmidb\S055\S055R08.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      

EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Used Annotations descriptions: ['T0', 'T1', 'T2']
405 matching events found
No baseline correction applied
Not setting metadata
Loading data for 405 events and 497 original time points ...
19 bad epochs dropped
(2347, 64, 497)
(2347,)


In [7]:
data_lenght=497 #lenght of data
n_ch=64 #number of channel

train_X=train_X.transpose(0,2,1)
train_X=np.reshape(train_X,[-1,data_lenght,n_ch,1])

#valid_X=valid_X.transpose(0,2,1)
#valid_X=np.reshape(valid_X,[-1,data_lenght,n_ch,1])

test_X=test_X.transpose(0,2,1)
test_X=np.reshape(test_X,[-1,data_lenght,n_ch,1])



train_X.shape

(2347, 497, 64, 1)

In [8]:
train_X= tf.keras.utils.normalize(train_X, axis=1,order=0)
#valid_X= tf.keras.utils.normalize(valid_X, axis=1,order=0)
test_X= tf.keras.utils.normalize(test_X, axis=1,order=0)

In [9]:
#Build the model

model = keras.Sequential([
    #conv_pool_block_1
    keras.layers.Conv2D(filters=25, kernel_size=(15,1),dilation_rate=(2, 1),strides=(1, 1), padding='valid', activation='elu', input_shape=(data_lenght,n_ch,1)),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),

    keras.layers.Conv2D(filters=25, kernel_size=(1,n_ch),strides=(1, 1), padding='valid', activation='elu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),
    keras.layers.MaxPool2D(pool_size=(3,1)),

    #conv_pool_block_2
    keras.layers.Conv2D(filters=50, kernel_size=(10,1),strides=(1, 1), padding='valid', activation='elu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),
    keras.layers.MaxPool2D(pool_size=(3,1)),
    
    #conv_pool_block_3
    keras.layers.Conv2D(filters=100, kernel_size=(10,1),strides=(1, 1), padding='valid', activation='elu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),
    keras.layers.MaxPool2D(pool_size=(3,1)),
    
    #conv_pool_block_4
    keras.layers.Conv2D(filters=200, kernel_size=(10,1),strides=(1, 1), padding='valid', activation='elu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),
    keras.layers.MaxPool2D(pool_size=(3,1)),

    #classification Layer
    keras.layers.Flatten(),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(2, activation='softmax')
    ])

# Take a look at the model summary
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 469, 64, 25)       400       
_________________________________________________________________
batch_normalization_v1 (Batc (None, 469, 64, 25)       100       
_________________________________________________________________
dropout (Dropout)            (None, 469, 64, 25)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 469, 1, 25)        40025     
_________________________________________________________________
batch_normalization_v1_1 (Ba (None, 469, 1, 25)        100       
_________________________________________________________________
dropout_1 (Dropout)  

In [10]:
#adam_my=keras.optimizers.Adam(lr=1 * 0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.10, amsgrad=False)


In [11]:
#from AdamW import AdamW
#batch_size=64
#epochs=30
#b, B, T = batch_size, train_X.shape[0], epochs
#wd = 0.005 * (b/B/T)**0.5
#adamw_my=AdamW(weight_decay=wd)

In [12]:
#Compile the model
#model.compile(optimizer=adam_my,
#              loss='binary_crossentropy',
#              metrics=['accuracy'])

In [13]:
#Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [14]:
#Train the model
model.fit(train_X, train_y, batch_size=64, epochs=30,validation_split=.10)#,callbacks=[])

Train on 2112 samples, validate on 235 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/30
2112/2112 [==============================] - 4s 2ms/sample - loss: 1.1542 - acc: 0.5289 - val_loss: 0.6858 - val_acc: 0.5149
Epoch 2/30
2112/2112 [==============================] - 2s 819us/sample - loss: 0.9159 - acc: 0.6122 - val_loss: 0.6837 - val_acc: 0.5234
Epoch 3/30
2112/2112 [==============================] - 2s 818us/sample - loss: 0.7794 - acc: 0.6383 - val_loss: 0.6351 - val_acc: 0.5702
Epoch 4/30
2112/2112 [==============================] - 2s 815us/sample - loss: 0.7275 - acc: 0.6638 - val_loss: 0.6452 - val_acc: 0.5702
Epoch 5/30
2112/2112 [==============================] - 2s 828us/sample - loss: 0.6715 - acc: 0.6851 - val_loss: 0.5726 - val_acc: 0.7574
Epoch 6/30
2112/2112 [==============================] - 2s 818us/sample - loss: 0.6453 - acc: 0.7036 - val_loss: 0.5532 - val_acc: 0.8255
Epoch 7/30
2112/2112 [==============================] - 2s 814us/sample - loss:

In [15]:
#test accuracy
test_loss, test_acc = model.evaluate(test_X,test_y)

print('\nTest accuracy:', test_acc)

386/386 [==============================] - 0s 631us/sample - loss: 0.7873 - acc: 0.7332

Test accuracy: 0.7331606


In [16]:
#Make predictions
predictions = model.predict(test_X)

In [17]:
predictions[0:10]>0.5

array([[False,  True],
       [ True, False],
       [False,  True],
       [ True, False],
       [False,  True],
       [ True, False],
       [False,  True],
       [ True, False],
       [ True, False],
       [False,  True]])

In [18]:
test_y[0:10]

array([1, 0, 1, 0, 1, 0, 1, 0, 0, 1], dtype=int64)